In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 1.x

In [ ]:
!pip install tiffile
!pip install elasticdeform
!pip install keras==2.2.5
!pip install csbdeep

In [ ]:
%cd '/content/drive/My Drive/FAQT/'
import sys
import os

import difflib
from csbdeep.utils import Path, normalize
import matplotlib.pyplot as plt
from skimage.morphology import remove_small_objects
sys.path.append('Utils')
import glob
import cv2
from scipy.ndimage.morphology import binary_fill_holes
from tifffile import imread, imwrite
from csbdeep.utils import Path, normalize

from TerminatorUtils.helpers import multiplot, doubleplot, WingArea
from csbdeep.models import Config, CARE
from skimage.filters import threshold_otsu


In [ ]:
Masterdir = '/content/drive/My Drive/Images_For_FAQT/'


LeftName = 'L'
RightName = 'R'

MaskLeftResults = Masterdir + '/MaskLeftResults/'
MaskRightResults = Masterdir + '/MaskRightResults/'

AsymmetryResults = Masterdir + '/AsymmetryResults/'
AsymmetryResultsName = 'Asymmetry'

Model_Dir = '/content/drive/My Drive/SegmentationModel/'

UNETSegmentationModelName = 'DeepWingSegmentationModelUNET'

UnetModel = CARE(config = None, name = UNETSegmentationModelName, basedir = Model_Dir)

In [ ]:
#Apply the prediction on left wing directory
min_size = 15000
Raw_path = os.path.join(Masterdir, '*tif')
Path(MaskResults).mkdir(exist_ok = True)
Path(AreaResults).mkdir(exist_ok = True)
axes = 'YXC'
saveaxes = 'YX'
filesRaw = glob.glob(Raw_path)
filesRaw.sort
count = 0
axis_norm = (0,1,2)   # normalize channels independently

AllArea = []
AllName = []
for fname in filesRaw:
          
            #Read image        
            image = imread(fname)
            originalX = image.shape[0]
            originalY = image.shape[1]  
            
            Name = os.path.basename(os.path.splitext(fname)[0])
         
            x = image
            
            Segmented = UnetModel.predict(x,axes)
            thresh = threshold_otsu(Segmented) 
            Binary = Segmented > thresh
            Filled = binary_fill_holes(Binary)
            NoSmall = remove_small_objects(Filled, min_size)
            Finalimage = CCLabels(NoSmall)
            RightArea, LeftArea, RightMinusLeft, RightPlusLeft, Assymetery = WingArea(Finalimage, Finalimage)
            AllArea.append(RightArea)
            AllName.append(Name)
            
            doubleplot(image,Finalimage[:,:,0], 'Original', 'UNET', plotTitle = 'Segmentation Result' )

            imwrite((MaskResultsRight + 'Mask' + Name + '.tif' ) , Finalimage.astype('uint8'))
            
           

In [ ]:
import pandas as pd
Path(AsymmetryResults).mkdir(exist_ok = True)
Raw_pathRight = os.path.join(MaskResults, '*tif')
Raw_pathLeft = os.path.join(MaskResults, '*tif')

filesRawRight = glob.glob(Raw_pathRight)
filesRawLeft = glob.glob(Raw_pathLeft)


AllRightArea = []
AllLeftArea = []
AllRightMinusLeftArea = []
AllRightPlusLeftArea = []
AllAssymetery = []
AllName = []
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
for fnameRight in filesRawRight:
    
   NameRight = os.path.basename(os.path.splitext(fnameRight)[0]) 
   imageRight = imread(fnameRight)
   for fnameLeft in filesRawLeft:
       NameLeft = os.path.basename(os.path.splitext(fnameLeft)[0]) 
       imageLeft = imread(fnameLeft) 
       
       ChangeName = difflib.ndiff(NameLeft, NameRight)
       delta = ''.join(x[0:] for x in ChangeName if x.startswith('- '))
      
       #ChangeName = NameRight.replace(RightName, LeftName) 
       if delta == '- L':
           print(NameLeft, NameRight)
           RightArea, LeftArea, RightMinusLeft, RightPlusLeft, Assymetery = WingArea(imageLeft, imageRight)
         
           AllName.append(NameLeft)
           AllRightArea.append(RightArea)
           AllLeftArea.append(LeftArea)
           AllRightMinusLeftArea.append(RightMinusLeft)
           AllRightPlusLeftArea.append(RightPlusLeft)
           AllAssymetery.append(Assymetery)
            
df = pd.DataFrame(list(zip(AllRightArea,AllLeftArea,AllRightMinusLeftArea,AllRightPlusLeftArea,AllAssymetery)), index =AllName, 
                                              columns =['RightArea', 'LeftArea', 'Right-Left', 'Right+Left', 'Assymmetery'])

df.to_csv(AsymmetryResults + '/' + AsymmetryResultsName +  '.csv')  
df

In [ ]:
positivecount = np.sum(df['Assymmetery']>0)
negativecount = np.sum(df['Assymmetery']<0)   
print('Positive Count' , positivecount)
print('Negative Count' , negativecount)

In [ ]:
multiplotline(AllRightArea, AllLeftArea, AllRightMinusLeftArea, AllRightPlusLeftArea, 'Right Area', 'Left Area', 'Right-Left', 'Right+Left', targetdir = AsymmetryResults, File = 'Areas', plotTitle = 'Area')

In [ ]:
plt.plot(AllAssymetery)
plt.title("Asymmetry")
plt.ylabel("Asymmetry")
plt.xlabel("Filenumber")
plt.show()